<a href="https://colab.research.google.com/github/folflora/AspNetCore.Docs/blob/main/PhotoProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install build-essential
!apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev
!apt-get install python-dev python-numpy libtbb2 libtbb-dev libjpeg-dev libpng-dev libtiff-dev libdc1394-22-dev

!cd /content/
!git clone https://github.com/opencv/opencv.git
!git clone https://github.com/opencv/opencv_contrib.git

!mkdir /content/opencv/build
%cd /content/opencv/build
#!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules -DWITH_CUDA=ON -DCMAKE_BUILD_TYPE=Release -D CMAKE_INSTALL_PREFIX=/usr/local -DBUILD_DOCS=TRUE -DBUIL_EXAMPLES=TRUE -DENABLE_FAST_MATH=1 -DCUDA_FAST_MATH=1 -DWITH_CUBLAS=1 ..
! time cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules -DWITH_CUDA=ON -DCMAKE_BUILD_TYPE=Release -D CMAKE_INSTALL_PREFIX=/usr/local -DBUILD_DOCS=FALSE -DBUIL_EXAMPLES=FALSE ..

! time make -j2
%cd /content/opencv/build
!time make install

A streamkimeneten csak az utolsó 5000 sor látható.
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_color_convert_all.c.o
[  1%] Building C object 3rdparty/ittnotify/CMakeFiles/ittnotify.dir/src/ittnotify/jitprofiling.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_color_convert_rgbs.c.o
[  1%] Linking C static library ../lib/libittnotify.a
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_filter_bilateral.c.o
[  1%] Built target ittnotify
[  1%] Building CXX object 3rdparty/protobuf/CMakeFiles/libprotobuf.dir/src/google/protobuf/any_lite.cc.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_filter_box.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_filter_canny.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_filter_gaussian.c.o
[  1%] Building C o

In [ ]:
%%writefile PhotoProcessing.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <iostream>
//#include <vector>
//#include <pngio.h>
//#include <time.h>
//#include <omp.h>

#include <opencv2/opencv.hpp>
#include <opencv2/core/core.hpp>
#include <opencv2/imgproc/imgproc.hpp>
#include <opencv2/imgcodecs.hpp>
#include <opencv2/highgui/highgui.hpp>


using namespace cv;
using namespace std;
//#include "Timing.h"

#define BLOCK_SIZE (16)
#define GRID_SIZE
#define FILTER_SIZE (5)
//#define TILE_SIZE () // BLOCK_SIZE - 2 * (FILTER_SIZE / 2)

const int TILE_SIZE = (BLOCK_SIZE - 2) * (FILTER_SIZE / 2);

// Defining kernel function
__global__ void processImage(unsigned char *out, unsigned char *in, 
size_t pitch, unsigned int width, unsigned int height)
{
  // Indices for the pixels we can reach
  int x_0 = TILE_SIZE * blockIdx.x + threadIdx.x;
  int y_0 = TILE_SIZE * blockIdx.y + threadIdx.y;

  // Indices to reach the border
  int x_i = x_0 - FILTER_SIZE / 2;
  int y_i = y_0 - FILTER_SIZE / 2;

  // Defining shared memory
  __shared__ unsigned char sBuffer[BLOCK_SIZE][BLOCK_SIZE];

  // Copying inside the shared memory
  if (x_i > 0 && x_i < width && y_i > 0 && y_i < height)
  {
    sBuffer[threadIdx.y][threadIdx.x] = in[y_i * pitch + x_i];
  } else 
  {
    sBuffer[threadIdx.y][threadIdx.x] = 0;
  }
  __syncthreads();
  
  int sum;
  if (threadIdx.x < TILE_SIZE && threadIdx.y < TILE_SIZE) 
  {
    sum = 0;
    for(int r = 0; r < FILTER_SIZE; r++) 
    {
      for (int c = 0; c < FILTER_SIZE; c++) 
      {
        sum += sBuffer[threadIdx.y + r][threadIdx.x + c];
      }
    }
    sum /= FILTER_SIZE * FILTER_SIZE;
    if (x_0 < width && y_0 < height)
    {
      out[y_0 * width + x_0 ] = sum;
    }    
  }
}

int main(int argc, char **argv) {  

  // Loading
  Mat image = imread("img_01.png");
  imshow("Actual image", image);//showing actual image//
  waitKey(0);//wait for key stroke
  destroyAllWindows();//closing all windows// 

  // Reading color channels
  Mat colour_channels[3];

  split(image, colour_channels); 
  Mat b = colour_channels[0];
  Mat g = colour_channels[1];
  Mat r = colour_channels[2];

  imshow("Blue Channel",b);//showing Blue channel//
  imshow("Green Channel",g);//showing Green channel//
  imshow("Red Channel",r);//showing Red channel//
  imshow("Actual image", image);//showing actual image//
  waitKey(0);//wait for key stroke
  destroyAllWindows();//closing all windows// 
  
  unsigned int width = image.size().width;
  unsigned int height = image.size().height;

  // Defining size to allocate memory
  unsigned int size = width * height * sizeof(unsigned char);

  // Allocate memory buffer to host memory
  unsigned char *h_r= (unsigned char*) malloc (size);
  unsigned char *h_g= (unsigned char*) malloc (size);
  unsigned char *h_b= (unsigned char*) malloc (size);

  // Allocate memory for the output
  unsigned char *h_r_n= (unsigned char*) malloc (size);
  unsigned char *h_g_n= (unsigned char*) malloc (size);
  unsigned char *h_b_n= (unsigned char*) malloc (size);

  // Converting image to raw buffer  
  h_r = r.data;
  h_g = g.data;
  h_b = b.data;

  // Allocate memory on the device
  unsigned char *d_r_n = NULL;  
  unsigned char *d_g_n = NULL;
  unsigned char *d_b_n = NULL;

  cudaMalloc(&d_r_n, size);
  cudaMalloc(&d_g_n, size);
  cudaMalloc(&d_b_n, size);

  unsigned char *d_r = NULL;
  unsigned char *d_g = NULL;
  unsigned char *d_b = NULL;

  size_t pitch_r = 0;
  size_t pitch_g = 0;
  size_t pitch_b = 0;
  
  cudaMallocPitch(&d_r, &pitch_r, width, height);
  cudaMallocPitch(&d_g, &pitch_g, width, height);
  cudaMallocPitch(&d_b, &pitch_b, width, height);

  // copy raw buffer from host to device 
  cudaMemcpy2D(d_r, pitch_r, h_r, pitch_r, width, height, cudaMemcpyHostToDevice);
  cudaMemcpy2D(d_g, pitch_g, h_g, pitch_g, width, height, cudaMemcpyHostToDevice);
  cudaMemcpy2D(d_b, pitch_b, h_b, pitch_b, width, height, cudaMemcpyHostToDevice);

  // configure image kernel
  //dim3 grid_size((width + BLOCK_SIZE - 1) / BLOCK_SIZE, (height + BLOCK_SIZE - 1) / BLOCK_SIZE));
  //dim3 block_size(BLOCK_SIZE, BLOCK_SIZE);
  dim3 grid_size((width + TILE_SIZE - 1)/TILE_SIZE, (height +TILE_SIZE- 1)/TILE_SIZE);
  dim3 block_size(BLOCK_SIZE, BLOCK_SIZE);

  //double start = omp_get_wtime();

  processImage<<<grid_size, block_size>>>(d_r_n, d_r, pitch_r, width, height);
  processImage<<<grid_size, block_size>>>(d_g_n, d_g, pitch_g, width, height);
  processImage<<<grid_size, block_size>>>(d_b_n, d_b, pitch_b, width, height);

  cudaDeviceSynchronize();

  //double end = omp_get_wtime();

  // copy from device to the host
  cudaMemcpy(h_r_n, d_r_n, size, cudaMemcpyDeviceToHost);
  cudaMemcpy(h_g_n, d_g_n, size, cudaMemcpyDeviceToHost);
  cudaMemcpy(h_b_n, d_b_n, size, cudaMemcpyDeviceToHost);

  // Convert raw buffer to png image

  Mat image_blurred = Mat::zeros(Size(image.rows, image.cols), CV_8UC1);
  //Mat red, green, blue;
  //Mat mod_channels[3] = { blue, green, red };
  colour_channels[0] = (int)*h_b_n;
  colour_channels[1] = (int)*h_g_n;
  colour_channels[2] = (int)*h_r_n;
  merge(colour_channels, 3, image_blurred);

  //pvg::rgb3ToPng(img, h_r_n, h_g_n, h_b_n);
  
  
  //std::cout<<"Done in"<<end-start<<" seconds"<<std::endl;

  imwrite("blurred.jpg", image_blurred);

  // Free memory
  cudaFree(d_r);
  cudaFree(d_r_n);
  
  cudaFree(d_g);
  cudaFree(d_g_n);

  cudaFree(d_b);
  cudaFree(d_b_n);

  free(h_r);
  free(h_r_n);
  
  free(h_g);
  free(h_g_n);

  free(h_b);
  free(h_b_n);

  return 0;
}


In [ ]:
!nvcc PhotoProcessing.cu -o PhotoProcessing

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!./PhotoProcessing

/bin/bash: ./PhotoProcessing: No such file or directory
